# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [4]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [5]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [6]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [7]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [8]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}

grid_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
grid_svd.fit(data)

In [9]:
# print out optimal parameters for SVD after GridSearch
print(grid_svd.best_score)
print(grid_svd.best_params)

{'rmse': 0.8698597244097709, 'mae': 0.6683443266439174}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [10]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [15]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(f"average RMSE score: {np.mean(cv_knn_basic['test_rmse'])}")

('test_rmse', array([0.97442004, 0.97350962, 0.96949851, 0.96529112, 0.97671392]))
('test_mae', array([0.75119335, 0.75337985, 0.75002439, 0.7465666 , 0.75405698]))
('fit_time', (0.6815407276153564, 0.6955626010894775, 0.6668961048126221, 0.6889684200286865, 0.6385226249694824))
('test_time', (0.7606101036071777, 0.7711553573608398, 0.7906513214111328, 0.7339017391204834, 0.7614610195159912))
-----------------------
average RMSE score: 0.9718866412640675


In [16]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [17]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)

print(f"average RMSE score: {np.mean(cv_knn_basic['test_rmse'])}")

('test_rmse', array([0.87446035, 0.87371953, 0.88147827, 0.87358335, 0.87624832]))
('test_mae', array([0.66365612, 0.66744939, 0.67382588, 0.66747224, 0.67323902]))
('fit_time', (0.49001049995422363, 0.4848053455352783, 0.5197417736053467, 0.5074565410614014, 0.5254943370819092))
('test_time', (1.0168070793151855, 0.9601056575775146, 0.9648683071136475, 1.0024445056915283, 1.0108520984649658))
average RMSE score: 0.9718866412640675


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [18]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [19]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [20]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [21]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.1539831113069225, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [36]:
def movie_rater(movie_df, num, genre=None):
    rating_list = []
    user_id = 420
    
    while num > 0:
        if genre:
            # provide a random movie of the same genre
            movie = movie_df[movie_df['genres'].str.contains(genre, case=False, na=False)].sample(1)
        else:
            # provide a random movie
            movie = movie_df.sample(1)
        
        print(f"\nMovie: {movie['title'].values[0]}")
        print(f"Genre: {movie['genres'].values[0]}")
        
        response = input("Rate this movie (1-5), or 's' to skip, 'q' to quit: ").lower()
        
        if response == 'q':
            return rating_list
        elif response == 's':
            continue
        elif response.isdigit() and 1 <= int(response) <= 5:
            rating = float(response)
            rating_list.append({
                'userId': user_id,
                'movieId': movie['movieId'].values[0],
                'rating': rating
            })
            num -= 1
        else:
            print("Invalid input. Please enter a number (1-5), 's' to skip, or 'q' to quit.")
    
    return rating_list

In [38]:
user_rating = movie_rater(df_movies, 5, 'Children')


Movie: Tall Tale (1995)
Genre: Adventure|Children|Fantasy|Western
Rate this movie (1-5), or 's' to skip, 'q' to quit: s

Movie: Balto (1995)
Genre: Adventure|Animation|Children
Rate this movie (1-5), or 's' to skip, 'q' to quit: 2

Movie: Jack Frost (1998)
Genre: Children|Comedy|Drama
Rate this movie (1-5), or 's' to skip, 'q' to quit: s

Movie: Cats Don't Dance (1997)
Genre: Animation|Children|Musical
Rate this movie (1-5), or 's' to skip, 'q' to quit: s

Movie: G.I. Joe: The Movie (1987)
Genre: Action|Adventure|Animation|Children|Fantasy|Sci-Fi
Rate this movie (1-5), or 's' to skip, 'q' to quit: 3

Movie: Flintstones, The (1994)
Genre: Children|Comedy|Fantasy
Rate this movie (1-5), or 's' to skip, 'q' to quit: 4

Movie: Jumanji: Welcome to the Jungle (2017)
Genre: Action|Adventure|Children
Rate this movie (1-5), or 's' to skip, 'q' to quit: 5

Movie: Next Karate Kid, The (1994)
Genre: Action|Children|Romance
Rate this movie (1-5), or 's' to skip, 'q' to quit: 5


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [39]:
user_rating

[{'userId': 420, 'movieId': 13, 'rating': 2.0},
 {'userId': 420, 'movieId': 26590, 'rating': 3.0},
 {'userId': 420, 'movieId': 355, 'rating': 4.0},
 {'userId': 420, 'movieId': 179401, 'rating': 5.0},
 {'userId': 420, 'movieId': 502, 'rating': 5.0}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [40]:
## add the new ratings to the original ratings DataFrame
user_ratings = pd.DataFrame(user_rating)
combined_ratings = pd.concat([new_df, user_ratings], axis=0)
new_data = Dataset.load_from_df(combined_ratings,reader)

In [41]:
# train a model using the new combined DataFrame
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(new_data.build_full_trainset())

In [42]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
user_id = 420

# Create list of tuples (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    predicted_rating = svd.predict(user_id, m_id).est
    list_of_movies.append((m_id, predicted_rating))

# Display the first few predictions to verify
print("First 5 predictions:")
print(list_of_movies[:5])

First 5 predictions:
[(1, 3.967696686779173), (3, 3.466051135422759), (6, 3.9611377868557422), (47, 4.023103959371645), (50, 4.335881427317897)]


In [43]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [44]:
# return the top n recommendations using the 
def recommended_movies(user_ratings, movie_title_df, n):
    print(f"Top {n} Movie Recommendations:")
    for i, (movie_id, predicted_rating) in enumerate(user_ratings[:n], 1):
        movie_info = movie_title_df[movie_title_df['movieId'] == movie_id].iloc[0]
        title = movie_info['title']
        genres = movie_info['genres']
        print(f"{i}. {title}")
        print(f"   Genres: {genres}")
        print(f"   Predicted Rating: {predicted_rating:.2f}")
        print()

recommended_movies(ranked_movies, df_movies, 5)

Top 5 Movie Recommendations:
1. Shawshank Redemption, The (1994)
   Genres: Crime|Drama
   Predicted Rating: 4.47

2. Ghost in the Shell (Kôkaku kidôtai) (1995)
   Genres: Animation|Sci-Fi
   Predicted Rating: 4.42

3. Lawrence of Arabia (1962)
   Genres: Adventure|Drama|War
   Predicted Rating: 4.41

4. Cool Hand Luke (1967)
   Genres: Drama
   Predicted Rating: 4.39

5. Streetcar Named Desire, A (1951)
   Genres: Drama
   Predicted Rating: 4.39



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.